In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras.layers import InputLayer
#from tensorflow.keras import layers,Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import os

In [2]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os
import sys

In [3]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.ResNet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import load_model

In [4]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
import numpy as np

In [5]:
from keras.layers import Dense, Activation, Flatten

In [6]:
base_dir = 'C:/Users/wanis'

In [7]:
train_ds  = os.path.join(base_dir, 'new train')
val_ds = os.path.join(base_dir, 'new validation')

In [8]:
batch_size = 20 #Try different values
epochs = 10
img_size = 224
img_height,img_width=224,224

In [9]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=5,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

In [10]:
resnet_model.add(Flatten())
resnet_model.add(Dense(128, activation='relu'))
resnet_model.add(Dense(7, activation='softmax'))

In [11]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dense_1 (Dense)             (None, 7)                 903       
                                                                 
Total params: 23,850,887
Trainable params: 263,175
Non-trainable params: 23,587,712
_________________________________________________________________


In [12]:
resnet_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

C:\Users\wanis\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [13]:
# define cnn model
def define_model():
	# load model
	model = tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=7,
                   weights='imagenet')
	# model = load_model('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
#	output = Dense(5, activation='sigmoid')(class1)
	output = Dense(7, activation='softmax')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
#	opt = SGD(lr=0.001, momentum=0.9)
	opt = tf.keras.optimizers.RMSprop(lr=0.0001)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

In [14]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
	
	# plot accuracy
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')
	
	# plot loss
    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,max(plt.ylim())])
    plt.title('Training and Validation Loss')
	
	# save plot to file
    filename = 'resnet_b20e10'
    plt.savefig(filename + '_plot.png')
    plt.close()

In [15]:
# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow_from_directory(train_ds,
		class_mode='categorical', batch_size=batch_size, target_size=(img_size, img_size))
	validate_it = datagen.flow_from_directory(val_ds,
		class_mode='categorical', batch_size=batch_size, target_size=(img_size, img_size))
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=validate_it, validation_steps=len(validate_it), epochs=epochs, verbose=1)
	# evaluate model
	acc = model.evaluate_generator(validate_it, steps=len(validate_it), verbose=0)
	acc = np.array(acc)
	print(acc[1]*100)    
    # save model
	model.save('C:/Users/wanis/resnetsoham.h5')
	print('Model saved as "resnetsoham.h5"')
	# learning curves
	summarize_diagnostics(history)

In [16]:
# entry point, run the test harness
run_test_harness()

C:\Users\wanis\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Found 700 images belonging to 7 classes.
Found 140 images belonging to 7 classes.


C:\Users\wanis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  app.launch_new_instance()


Epoch 1/10
35/35 [==============================] - 88s 2s/step - loss: 0.3691 - accuracy: 0.4943 - val_loss: 0.2620 - val_accuracy: 0.7286
Epoch 2/10
35/35 [==============================] - 86s 2s/step - loss: 0.2202 - accuracy: 0.8314 - val_loss: 0.1719 - val_accuracy: 0.8571
Epoch 3/10
35/35 [==============================] - 85s 2s/step - loss: 0.1547 - accuracy: 0.8957 - val_loss: 0.1333 - val_accuracy: 0.9143
Epoch 4/10
35/35 [==============================] - 89s 3s/step - loss: 0.1182 - accuracy: 0.9243 - val_loss: 0.1089 - val_accuracy: 0.9143
Epoch 5/10
35/35 [==============================] - 97s 3s/step - loss: 0.0952 - accuracy: 0.9400 - val_loss: 0.0954 - val_accuracy: 0.9071
Epoch 6/10
35/35 [==============================] - 101s 3s/step - loss: 0.0801 - accuracy: 0.9471 - val_loss: 0.0871 - val_accuracy: 0.9143
Epoch 7/10
35/35 [==============================] - 101s 3s/step - loss: 0.0682 - accuracy: 0.9529 - val_loss: 0.0795 - val_accuracy: 0.9143
Epoch 8/10
35/35 [

C:\Users\wanis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.


92.85714030265808


C:\Users\wanis\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Model saved as "resnetsoham.h5"


In [17]:
######
## Extract Features:
# model_feature = load_model('VGG16.h5')
model_feature = tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=5,
                   weights='imagenet')
model_feature.summary()

import pandas as pd
train_img = os.listdir(train_ds)
images = []
resnet_feature_list = []
label = []
# print(train_img)
# exit()
for folder in train_img:
#    print(folder)
    for filename in os.listdir('new train/'+folder):
        label.append(folder)
#        print('new train/'+folder+'/'+filename)
#        im = os.path.join('train',filename)
        img_path = 'new train/'+folder+'/'+filename
        images.append(img_path)
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        resnet_feature = model_feature.predict(img_data)
        resnet_feature_np = np.array(resnet_feature)
        resnet_feature_list.append(resnet_feature_np.flatten())
        
resnet_feature_list_np = np.array(resnet_feature_list)

df_feature = pd.DataFrame(list(zip(images,resnet_feature_list_np,label)),columns= ['img_path','feature_vector','label'])

df_feature.to_pickle('resnet data feature 700')

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [18]:
###
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_labels():
	datagen = ImageDataGenerator(featurewise_center=True)
	
	train_it = datagen.flow_from_directory(train_ds,class_mode='categorical', batch_size=batch_size, target_size=(img_size, img_size))
#	y_true_labels = train_it.classes
	label_map = (train_it.class_indices)
	print('Class labels: ',label_map)

get_labels()

Found 700 images belonging to 7 classes.
Class labels:  {'blazer_men': 0, 'blazer_women': 1, 'dress': 2, 'hoodie': 3, 'jeans_men': 4, 'jeans_women': 5, 'shorts': 6}


In [19]:
from PIL import Image

In [20]:
###
# Resize images
path = "C:/Users/wanis/new train"
folder = os.listdir(path)

path2 = "C:/Users/wanis/v4/static/new train"

def resize():
	for one in folder:
	    dirs = os.listdir(path+'/'+one)
	    for item in dirs:
	       if os.path.isfile(path+'/'+one+'/'+item):
	           im = Image.open(path+'/'+one+'/'+item)
	           
	           imResize = im.resize((200,300), Image.ANTIALIAS)
	           imResize.save(path2+'/'+one+'/'+item, 'JPEG', quality=90)

resize()


In [21]:
# Extract feature vectors of train data

import pandas as pd

base_dir = 'C:/Users/wanis'

train_dir = os.path.join(base_dir, 'new train')
validation_dir = os.path.join(base_dir, 'new validation')

train_img = os.listdir(train_dir)
images = []
resnet_feature_list = []
label = []

model_feature = tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=5,
                   weights='imagenet')
# model_feature = load_model('VGG16.h5')

In [22]:
for folder in train_img:
#    print(folder)
    for filename in os.listdir(base_dir+'/new train/'+folder):
        label.append(folder)
#        print('train/'+folder+'/'+filename)
#        im = os.path.join('train',filename)
        img_path = base_dir+'/new train/'+folder+'/'+filename
        images.append(img_path)
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        resnet_feature = model_feature.predict(img_data)
        resnet_feature_np = np.array(resnet_feature)
        resnet_feature_list.append(resnet_feature_np.flatten())

In [23]:
resnet_feature_list_np = np.array(resnet_feature_list)

In [24]:
df_feature = pd.DataFrame(list(zip(images,resnet_feature_list_np,label)),columns= ['img_path','feature_vector','label'])

In [25]:
df_feature.to_pickle('resnet data feature 700')